# ShoeCo Problem

ShoeCo needs to plan production for the next 4 months. At the beginning of the first month, ShoeCo has 500 pairs of shoes in inventory. The demand forecast for each of the next 4 months is 

Month: |    1 |    2 |    3 |    4 |
 -----:|-----:|-----:|-----:|-----:|
Demand:| 3000 | 5000 | 2000 | 1000 |


ShoeCo has 100 workers currently employed who work 160 hours/month. Each worker is paid \$1500/month. They can also work overtime for up to 20 hours/worker each month. Overtime is paid at \$13/hour.  ShoeCo can choose to hire a new worker for \$1600 or fire a worker for \$2000. To produce a pair of shoes, 4 hours of labor and \$15 of raw materials are required. ShoeCo must meet demand every month and must pay \$3 for every pair of shoes in inventory at the end of each month. How should ShoeCo produce for the next 4 months if they are trying to minimize their total costs?


## Model and Solution in Julia

Note this is NOT a general model. All parameters are explicitly given as numbers in the model. This isn't a good way to write a model, but it's much easier to understand what's going on in the example this way!

In [13]:
using JuMP, Clp

d = [3000 5000 2000 1000] # monthly shoe demand
 
m = Model(solver=ClpSolver())

@variable(m, x[1:4] >= 0 ) # shoes produced in month t=1,2,3,4
@variable(m, w[1:4] >= 0 ) # workers employed in month t=1,2,3,4
@variable(m, o[1:4] >= 0 ) # overtime hours in month t=1,2,3,4
@variable(m, h[1:4] >= 0 ) # workers hired in month t=1,2,3,4
@variable(m, f[1:4] >= 0 ) # workers fired in month t=1,2,3,4
@variable(m, i[1:4] >= 0 ) # shoes in inventory in month t=1,2,3,4

@objective(m, Min, 15*sum(x) + 16*sum(o) + 1600*sum(h) + 2000*sum(f)
    + 1500*sum(w) + 3*sum(i))

@constraint(m, production[t in 1:4], 4*x[t] <= 160*w[t] + o[t])
@constraint(m, overtime[t in 1:4], o[t] <= 20*w[t])

@constraint(m, inv_bal_init, 500 + x[1] == d[1] + i[1])
@constraint(m, inv_bal[t in 2:4], i[t-1] + x[t] == d[t] + i[t])

@constraint(m, work_bal_init, 100 - f[1] + h[1] == w[1])
@constraint(m, work_bal[t in 2:4], w[t-1] - f[t] + h[t] == w[t])

solve(m)

println("Build ", Array(getvalue(x')), " shoes each month")
println("Use ", Array(getvalue(w')), " workers each month")
println("Use ", Array(getvalue(o')), " overtime hours each month")
println("Cost: ", getobjectivevalue(m))

Build [3750.0 3750.0 2000.0 1000.0] shoes each month
Use [93.75 93.75 50.0 50.0] workers each month
Use [0.0 0.0 0.0 0.0] overtime hours each month
Cost: 692500.0


## Post-solution analysis
Notice any problems with this solution?
What are some ways we could "fix" this?